In [ ]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import geemap
import ee
from google.colab import drive
drive.mount('/content/drive/')

Map = geemap.Map()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
roi = ee.Geometry.Polygon([[
            [79.5,42.5],
            [88,42.5],
            [88,46.5],
            [79.5,46.5],
            [79.5,42.5],]])

In [ ]:
Climate_collection = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filter(ee.Filter.date('2016-01-01', '2021-12-31')).select(['pr','tmmn','tmmx','srad'])
MOD09GA_collection_fir2y = ee.ImageCollection('MODIS/061/MOD09GA').filter(ee.Filter.date('2016-01-01', '2021-12-31'))
#first several years
#MOD09GA_collection_lateryears = ee.ImageCollection('MODIS/061/MOD09GA').filter(ee.Filter.date('2002-08-08', '2005-12-31'))
#later years
MOD09GA_collection_lateryears = ee.ImageCollection('MODIS/061/MOD09GA').filter(ee.Filter.date('2016-01-01', '2021-12-31'))
MYD09GA_collection = ee.ImageCollection('MODIS/061/MYD09GA').filter(ee.Filter.date('2016-01-01', '2021-12-31'))
Landcover_collection = ee.ImageCollection('MODIS/061/MCD12Q1').select('LC_Type1')

In [ ]:
def reprojectclip(image):
    return image.reproject(crs='EPSG:4326',scale=500).clip(roi)

def solar_radiation_qc(image):
    srad = image.select('srad').multiply(0.1)
    tmmn = image.select('tmmn').multiply(0.1)
    tmmx = image.select('tmmx').multiply(0.1)
    pr = image.select('pr')
    tmean = tmmn.add(tmmx).divide(2).rename('tmean')
    datestr = image.get('system:time_start')
    year = ee.Date(datestr).get('year')
    month = ee.Date(datestr).get('month')
    I_month = tmean.divide(5).pow(1.514).rename('I_month')

    return image.addBands(srad,None,True).addBands(tmmn,None,True).addBands(tmmx,None,True).addBands(pr,None,True).addBands(tmean).addBands(I_month).reproject(crs='EPSG:4326',scale=500).clip(roi).set('year',year).set('month',month).set('yearmonth',ee.Date(datestr).format('YYYYMM'))

Climate_collection = Climate_collection.map(solar_radiation_qc)
MOD09GA_collection_fir2y = MOD09GA_collection_fir2y.map(reprojectclip)
MOD09GA_collection_lateryears = MOD09GA_collection_lateryears.map(reprojectclip)
MYD09GA_collection = MYD09GA_collection.map(reprojectclip)
Landcover_collection = Landcover_collection.map(reprojectclip)

In [ ]:
def bitwiseExtract(image,frombit,tobit):
    maskSize = ee.Number(1).add(tobit).subtract(frombit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return image.rightShift(frombit).bitwiseAnd(mask)

def MOD09GA_QC_FC(image):
    qc_500_MOD = image.select('state_1km_MOD')
    cloud_state_MOD = bitwiseExtract(qc_500_MOD,0,1).eq(0)
    cloud_shadow_MOD = bitwiseExtract(qc_500_MOD,2,2).eq(0)
    cirrus_MOD = bitwiseExtract(qc_500_MOD,8,9).lte(1)
    cloud_algorithm_MOD = bitwiseExtract(qc_500_MOD,10,10).eq(0)
    mask_MOD = cloud_state_MOD.And(cloud_shadow_MOD).And(cirrus_MOD).And(cloud_algorithm_MOD)
    MODBands = image.select('sur_refl_MOD_b0.').updateMask(mask_MOD).multiply(0.0001)

    qc_500_MYD = image.select('state_1km_MYD')
    cloud_state_MYD = bitwiseExtract(qc_500_MYD,0,1).eq(0)
    cloud_shadow_MYD = bitwiseExtract(qc_500_MYD,2,2).eq(0)
    cirrus_MYD = bitwiseExtract(qc_500_MYD,8,9).lte(1)
    cloud_algorithm_MYD = bitwiseExtract(qc_500_MYD,10,10).eq(0)
    mask_MYD = cloud_state_MYD.And(cloud_shadow_MYD).And(cirrus_MYD).And(cloud_algorithm_MYD)
    MYDBands = image.select('sur_refl_MYD_b0.').updateMask(mask_MYD).multiply(0.0001)

    return image.addBands(MODBands,None,True).addBands(MYDBands,None,True)

def MOD09GA_QC(image):
    qc_500 = image.select('state_1km')
    cloud_state = bitwiseExtract(qc_500,0,1).eq(0)
    cloud_shadow = bitwiseExtract(qc_500,2,2).eq(0)
    cirrus = bitwiseExtract(qc_500,8,9).lte(1)
    cloud_algorithm = bitwiseExtract(qc_500,10,10).eq(0)
    mask = cloud_state.And(cloud_shadow).And(cirrus).And(cloud_algorithm)
    Bands = image.select('sur_refl_b0.').updateMask(mask).multiply(0.0001)

    return image.addBands(Bands,None,True)

In [ ]:
innerJoin = ee.Join.inner()
filterTimeEq = ee.Filter.equals(**{
    'leftField':'system:time_start',
    'rightField':'system:time_start'
})

def Cal_terra_aqua_average_value(terra,aqua):
    data = terra.unmask(0).add(aqua.unmask(0))
    mask = terra.mask().add(aqua.mask())
    mask_for_multiply = mask.where(mask.eq(0),10).pow(-1)
    mask_for_update = mask.where(mask.gt(0),1)
    average_value = data.multiply(mask_for_multiply)
    average_value = average_value.updateMask(mask_for_update)

    return average_value

In [ ]:
MOD09GA = MOD09GA_collection_lateryears.select(['sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07','state_1km'],['sur_refl_MOD_b01','sur_refl_MOD_b02','sur_refl_MOD_b03','sur_refl_MOD_b04','sur_refl_MOD_b05','sur_refl_MOD_b06','sur_refl_MOD_b07','state_1km_MOD'])
MYD09GA = MYD09GA_collection.select(['sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07','state_1km'],['sur_refl_MYD_b01','sur_refl_MYD_b02','sur_refl_MYD_b03','sur_refl_MYD_b04','sur_refl_MYD_b05','sur_refl_MYD_b06','sur_refl_MYD_b07','state_1km_MYD'])

innerJoinedMODLST = innerJoin.apply(MOD09GA, MYD09GA, filterTimeEq)
MODIS09GA = ee.ImageCollection(innerJoinedMODLST.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

MODIS09GA_qc = MODIS09GA.map(MOD09GA_QC_FC).select(['sur_refl_MOD_b01','sur_refl_MOD_b02','sur_refl_MOD_b03','sur_refl_MOD_b04','sur_refl_MOD_b05','sur_refl_MOD_b06','sur_refl_MOD_b07','state_1km_MOD','sur_refl_MYD_b01','sur_refl_MYD_b02','sur_refl_MYD_b03','sur_refl_MYD_b04','sur_refl_MYD_b05','sur_refl_MYD_b06','sur_refl_MYD_b07'])
MOD09GA_collection_fir2y_qc = MOD09GA_collection_fir2y.map(MOD09GA_QC).select(['sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07'],['MODIS_Band1','MODIS_Band2','MODIS_Band3','MODIS_Band4','MODIS_Band5','MODIS_Band6','MODIS_Band7'])

In [ ]:
def sameday_average(image):
    MOD_Band1 = image.select('sur_refl_MOD_b01')
    MYD_Band1 = image.select('sur_refl_MYD_b01')
    MOD_Band2 = image.select('sur_refl_MOD_b02')
    MYD_Band2 = image.select('sur_refl_MYD_b02')
    MOD_Band3 = image.select('sur_refl_MOD_b03')
    MYD_Band3 = image.select('sur_refl_MYD_b03')
    MOD_Band4 = image.select('sur_refl_MOD_b04')
    MYD_Band4 = image.select('sur_refl_MYD_b04')
    MOD_Band5 = image.select('sur_refl_MOD_b05')
    MYD_Band5 = image.select('sur_refl_MYD_b05')
    MOD_Band6 = image.select('sur_refl_MOD_b06')
    MYD_Band6 = image.select('sur_refl_MYD_b06')
    MOD_Band7 = image.select('sur_refl_MOD_b07')
    MYD_Band7 = image.select('sur_refl_MYD_b07')

    averaged_Band1 = Cal_terra_aqua_average_value(MOD_Band1,MYD_Band1)
    averaged_Band2 = Cal_terra_aqua_average_value(MOD_Band2,MYD_Band2)
    averaged_Band3 = Cal_terra_aqua_average_value(MOD_Band3,MYD_Band3)
    averaged_Band4 = Cal_terra_aqua_average_value(MOD_Band4,MYD_Band4)
    averaged_Band5 = Cal_terra_aqua_average_value(MOD_Band5,MYD_Band5)
    averaged_Band6 = Cal_terra_aqua_average_value(MOD_Band6,MYD_Band6)
    averaged_Band7 = Cal_terra_aqua_average_value(MOD_Band7,MYD_Band7)

    return image.addBands(averaged_Band1.rename('MODIS_Band1')).addBands(averaged_Band2.rename('MODIS_Band2')).\
            addBands(averaged_Band3.rename('MODIS_Band3')).addBands(averaged_Band4.rename('MODIS_Band4')).\
            addBands(averaged_Band5.rename('MODIS_Band5')).addBands(averaged_Band6.rename('MODIS_Band6')).\
            addBands(averaged_Band7.rename('MODIS_Band7')).toFloat()

In [ ]:
MODIS09GA_average = MODIS09GA_qc.map(sameday_average).select(['MODIS_Band1','MODIS_Band2','MODIS_Band3','MODIS_Band4','MODIS_Band5','MODIS_Band6','MODIS_Band7'])
def avoid_data_type_error(image):
    return image.toFloat()

#first several years
#MOD09GA_collection_fir2y_qc = MOD09GA_collection_fir2y_qc.map(avoid_data_type_error)
#MODIS09GA_Daily_averaged = MOD09GA_collection_fir2y_qc.merge(MODIS09GA_average)
#later years
MODIS09GA_Daily_averaged = MODIS09GA_average

In [ ]:
landcover2022 = ee.Image('MODIS/061/MCD12Q1/2021_01_01').select('LC_Type1').set('system:time_start',1640995200000).set('system:time_end',1672531200000)
landcover_final = Landcover_collection.merge(landcover2022)

In [ ]:
def generate_LUE(image):
    image = image.where(image.eq(3),0.485).where(image.eq(1),0.389).\
                where(image.eq(4),0.692).where(image.eq(2),0.985).\
                where(image.eq(5),0.768).where(image.eq(6),0.429).\
                where(image.eq(7),0.429).where(image.eq(8),0.542).\
                where(image.eq(9),0.542).where(image.eq(10),0.542).\
                where(image.eq(11),0.542).where(image.eq(12),0.542).\
                where(image.eq(13),0.542).where(image.eq(14),0.542).\
                where(image.eq(15),0.542).where(image.eq(16),0.542).where(image.eq(17),0.542)
    return image.reproject(crs='EPSG:4326',scale=500).clip(roi).toFloat()

def set_NDVImin(image):
    image = image.where(image.gt(0),0.023)
    return image.reproject(crs='EPSG:4326',scale=500).clip(roi).toFloat()

def set_NDVImax(image):
    image = image.where(image.eq(3),0.738).where(image.eq(1),0.647).\
                where(image.eq(2),0.676).where(image.eq(4),0.747).\
                where(image.eq(6),0.636).where(image.eq(7),0.636).\
                where(image.eq(8),0.636).where(image.eq(9),0.636).\
                where(image.gt(9),0.634).where(image.eq(5),0.702)
    return image.reproject(crs='EPSG:4326',scale=500).clip(roi).toFloat()

def set_SRmin(image):
  image = image.where(image.gt(0),1.05)
  return image.reproject(crs='EPSG:4326',scale=500).clip(roi).toFloat()

def set_SRmax(image):
  image = image.where(image.eq(3),6.63).where(image.eq(1),4.67).\
                where(image.eq(2),5.17).where(image.eq(4),6.91).\
                where(image.eq(6),4.49).where(image.eq(7),4.49).\
                where(image.eq(8),4.49).where(image.eq(9),4.49).\
                where(image.gt(9),4.44).where(image.eq(5),5.85)
  return image.reproject(crs='EPSG:4326',scale=500).clip(roi).toFloat()


In [ ]:
LUE = landcover_final.map(generate_LUE).select(['LC_Type1'],['LUE'])
NDVImin = landcover_final.map(set_NDVImin).select(['LC_Type1'],['NDVI_min'])
NDVImax = landcover_final.map(set_NDVImax).select(['LC_Type1'],['NDVI_max'])
SRmin = landcover_final.map(set_SRmin).select(['LC_Type1'],['SR_min'])
SRmax = landcover_final.map(set_SRmax).select(['LC_Type1'],['SR_max'])

In [ ]:
def Cal_index(image):
    red = image.select('MODIS_Band1')
    Nir = image.select('MODIS_Band2')
    Blue = image.select('MODIS_Band3')
    Green = image.select('MODIS_Band4')
    Swir = image.select('MODIS_Band6')

    NDVI = Nir.subtract(red).divide(Nir.add(red))
    NDVI = NDVI.where(NDVI.gt(1),1).where(NDVI.lt(-1),-1)
    NDWI = Green.subtract(Nir).divide(Green.add(Nir))
    NDWI = NDWI.where(NDWI.gt(1),1).where(NDWI.lt(-1),-1)
    NDBI = Swir.subtract(Nir).divide(Swir.add(Nir))
    NDBI = NDVI.where(NDBI.gt(1),1).where(NDBI.lt(-1),-1)
    NIRV = NDVI.multiply(Nir)
    SR = ee.Image(1).add(NDVI).divide(ee.Image(1).subtract(NDVI))

    return image.addBands(NDVI.rename('NDVI')).addBands(NDWI.rename('NDWI')).addBands(NDBI.rename('NDBI')).addBands(NIRV.rename('NIRV')).addBands(SR.rename('SR'))

MODIS09_indices = MODIS09GA_Daily_averaged.map(Cal_index).select(['NDVI','NDWI','NDBI','NIRV','SR'])

In [ ]:
def setyear(image):
    year_str = image.get('system:time_start')
    year = ee.Date(year_str).get('year')
    month = ee.Date(year_str).get('month')
    return image.set('year',year).set('month',month)

LUE = LUE.map(setyear)
NDVImin = NDVImin.map(setyear)
NDVImax = NDVImax.map(setyear)
SRmin = SRmin.map(setyear)
SRmax = SRmax.map(setyear)
MODIS09_indices = MODIS09_indices.map(setyear)

In [ ]:
filterTimeEqminmax = ee.Filter.equals(**{
    'leftField':'year',
    'rightField':'year'
})

innerJoinedIL_LUE = innerJoin.apply(MODIS09_indices, LUE, filterTimeEqminmax)
Indices_LUE = ee.ImageCollection(innerJoinedIL_LUE.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

innerJoinedILL_Nm = innerJoin.apply(Indices_LUE, NDVImin, filterTimeEqminmax)
Indices_LLM = ee.ImageCollection(innerJoinedILL_Nm.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

innerJoinedILL_NN = innerJoin.apply(Indices_LLM, NDVImax, filterTimeEqminmax)
Indices_LLM = ee.ImageCollection(innerJoinedILL_NN.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

innerJoinedILL_NNS = innerJoin.apply(Indices_LLM, SRmin, filterTimeEqminmax)
Indices_LLM = ee.ImageCollection(innerJoinedILL_NNS.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

innerJoinedILL_NNSR = innerJoin.apply(Indices_LLM, SRmax, filterTimeEqminmax)
Indices_LLM = ee.ImageCollection(innerJoinedILL_NNSR.map(lambda feature: ee.Image.cat(feature.get('primary'), feature.get('secondary'))))

In [ ]:
start = ee.Date('2016-01-01')
end = ee.Date('2022-01-31')
n_months = end.difference(start, 'month').subtract(1)
months = ee.List.sequence(0, n_months).map(lambda n : start.advance(n, 'month'))

# def makeMonthlyComposite_NDVI(date):
#     date = ee.Date(date)
#     return NDVI_LST.filterDate(date, date.advance(1, 'month')).mean().set("system:index", date.format("YYYYMM")).set("year", date.format("YYYY")).set("month", date.format("MM"))
T = Climate_collection.filter(ee.Filter.eq('month', 7)).select(['tmean'],['T'])

def getTopt(image):
  tmean = image.select('T')

  Topt_mean = ee.Number(tmean.reduceRegion(**{
            'reducer': ee.Reducer.mean(),
            'geometry' :roi          'scale': 500,
            'maxPixels': 1e9
            }).values().get(0))

  Topt = ee.Image(Topt_mean).rename('Topt')

  return image.addBands(Topt).reproject(crs='EPSG:4326',scale=500).clip(roi)

T = T.map(getTopt).select('Topt')

In [ ]:
I = Climate_collection.select('I_month')
n_years = end.difference(start, 'year').subtract(1)
years = ee.List.sequence(0, n_years).map(lambda n : start.advance(n, 'year'))

def makeYearlyComposite_I(date):
  date = ee.Date(date)
  return I.filterDate(date, date.advance(1, 'year')).sum().set('system:index',date.format("YYYY")).set('year',date.get('year')).set('system:time_start',date.millis()).set('system:time_end',date.advance(1, 'year').millis())

I = ee.ImageCollection.fromImages(years.map(makeYearlyComposite_I)).select(['I_month'],['I'])
I = I.map(reprojectclip)

In [ ]:
def makeMonthlyComposite_MOD09(date):
    date = ee.Date(date)
    return Indices_LLM.filterDate(date, date.advance(1, 'month')).max().set("system:index", date.format("YYYYMM")).set("year", date.format("YYYY")).set("system:time_start", date.millis()).set("system:time_end", date.advance(1, 'month').millis())

Indices_LLM = ee.ImageCollection.fromImages(months.map(makeMonthlyComposite_MOD09))
MODIS_SR = Climate_collection.combine(Indices_LLM)

innerJoinedTopt = innerJoin.apply(MODIS_SR, T, filterTimeEqminmax)
MODIS_SR = ee.ImageCollection(innerJoinedTopt.map(lambda  feature : ee.Image.cat(feature.get('primary'),feature.get('secondary'))))

innerJoinedClim = innerJoin.apply(MODIS_SR, I, filterTimeEqminmax)
MODIS_SR = ee.ImageCollection(innerJoinedClim.map(lambda  feature : ee.Image.cat(feature.get('primary'),feature.get('secondary'))))

In [ ]:
def Calstress(image):
  I = image.select('I')
  tmean = image.select('tmean')
  pre = image.select('pr')
  Topt = image.select('Topt')

  a = ee.Image().expression(
    '(0.6751 * I ** 3 - 77.1 * I ** 2 + 17920 * I + 482390) / 1000000',{'I':I}
    )

  Ep0 = ee.Image().expression(
    '16 * (10 * d / b) ** c',{'d':tmean,'b':I,'c':a}
    )

  Rn = ee.Image().expression(
    '((a * b) ** 0.5) * (0.369 + 0.598 * (a / b) ** 0.5)',{'a':Ep0,'b':pre}
    )

  EET = ee.Image().expression(
    'a * b * (a ** 2 + b ** 2 + a + b) / ((a + b) * (a ** 2 + b ** 2))',{'a':pre,'b':Rn}
    )

  PET = ee.Image().expression(
    '(a + b) / 2',{'a':EET,'b':Ep0}
    )

  Wstress = ee.Image().expression(
    '0.5 + 0.5 * (a / b)',{'a':EET,'b':PET}
    ).rename('Wstress')

  Tstress1 = ee.Image().expression(
    '0.8 + 0.02 * a - 0.0005 * a ** 2',{'a':Topt}
    )
  Tstress1 = Tstress1.where(tmean.lt(-10),0).rename('Tstress1')

  Tsigma2 = ee.Image().expression(
    '1.184 / (1 + exp(0.2 * (a - 10 - b))) * 1 / (1 + exp(0.3 * (b - a - 10)))',{'a':Topt,'b':tmean}
    )
  Tsigma3 = ee.Image(1.184).divide(ee.Image(1).add(ee.Image(-2).exp())).multiply(ee.Image(1).divide(ee.Image(1).add(ee.Image(-3).exp())))
  diff = tmean.subtract(Topt)
  diff1 = diff.where(diff.lt(-13),0).where(diff.gte(-13).And(diff.lte(10)),1).where(diff.gt(10),0)
  diff2 = diff.where(diff.gte(-13).And(diff.lte(10)),1).where(diff.gt(10),0).where(diff.lt(-13),0)
  Tstress2 = Tsigma2.multiply(diff1).add(Tsigma3.multiply(diff2)).rename('Tstress2')

  return image.addBands(Wstress).addBands(Tstress1).addBands(Tstress2).clip(roiODIS_SR = MODIS_SR.map(Calstress)

In [ ]:
def CalNPP(image):
  NDVI = image.select('NDVI')
  SR = image.select('SR')
  NDVImin = image.select('NDVI_min')
  NDVImax = image.select('NDVI_max')
  SRmin = image.select('SR_min')
  SRmax = image.select('SR_max')
  FPAR1 = NDVI.subtract(NDVImin).divide(NDVImax.subtract(NDVImin)).multiply(ee.Image(0.949)).add(0.001)
  FPAR2 = SR.subtract(SRmin).divide(SRmax.subtract(SRmin)).multiply(ee.Image(0.949)).add(0.001)
  FPAR = FPAR1.multiply(0.5).add(FPAR2.multiply(0.5))
  FPAR = FPAR.where(FPAR.gt(0.95),0.95).where(FPAR.lt(0.05),0.05).rename('FPAR')

  Sol = image.select('srad')
  APAR = FPAR.multiply(Sol).multiply(ee.Image(0.5)).rename('APAR')
  LUE = image.select('LUE')
  Wstress = image.select('Wstress')
  Tstress1 = image.select('Tstress1')
  Tstress2 = image.select('Tstress2')
  yupson = Tstress1.multiply(Tstress2).multiply(Wstress).multiply(LUE).rename('yupson')
  NPP = APAR.multiply(yupson).rename('NPP')
  return image.addBands(NPP).addBands(yupson).addBands(APAR).toFloat()

MODIS_SR = MODIS_SR.map(CalNPP)
NPP_month = MODIS_SR.select('NPP')

def makeYearlyComposite_NPP(date):
  date = ee.Date(date)
  return NPP_month.filterDate(date, date.advance(1, 'year')).sum().set('system:index',date.format("YYYY")).set('year',date.get('year')).set('system:time_start',date.millis()).set('system:time_end',date.advance(1, 'year').millis())

NPP_Year = ee.ImageCollection.fromImages(years.map(makeYearlyComposite_NPP))
# print(NPP_Year.getInfo())

In [ ]:
# Map.addLayer(MODIS_SR.select('yupson').filter(ee.Filter.date('2001-01-01', '2002-01-01')),{},'LUE')
# Map.addLayer(MODIS_SR.select('APAR').filter(ee.Filter.date('2001-01-01', '2002-01-01')),{},'APAR')
# Map

In [ ]:
geemap.ee_export_image_collection_to_drive(NPP_month, folder='NPP_roi', scale=500, region=roi)
geemap.ee_export_image_collection_to_drive(NPP_Year, folder='NPP_roi', scale=500, region=roi)

Total number of images: 72

Total number of images: 6

